In [ ]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from os.path import join, exists
import open3d as o3d
import matplotlib.pyplot as plt
from itertools import combinations
import copy
from tabulate import tabulate

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
import sys
sys.path.append('../')

from utils import *

In [ ]:
def draw_improvement(sim_old, sim_new, fpc, scale=None):
    improvement = sim_new - sim_old
    
    if scale is None:
        scale = 2 * torch.max(improvement.max(), -improvement.min())
    
    print(f'Scaled by {scale}')
    improvement = improvement / (2*scale) + 0.5

    # heatmap
    cmap = plt.get_cmap('bwr')
    
    colors = cmap(improvement.detach().cpu().numpy().squeeze())
    pcd_heatmap = o3d.geometry.PointCloud()
    pcd_heatmap.points = o3d.utility.Vector3dVector(np.asarray(fpc))
    pcd_heatmap.colors = o3d.utility.Vector3dVector(colors[:, :3])
    
    #o3d.visualization.draw_plotly([pcd_heatmap])
    o3d.visualization.draw_geometries([pcd_heatmap])

def highlight_query(query, feature_type, agg_type, distill, fused, fpc, fpcc, device, draw=True, scale=1, 
                    quantile=0.5, norm=False):
    
    import clip
    model, preprocess = clip.load("ViT-L/14@336px")
    
    with torch.no_grad():
        per_descriptor_embeds = []
        for descriptor in tqdm(query):
            _prompt = descriptor
            print(_prompt)
            texts = clip.tokenize(_prompt)  #tokenize
            texts = texts.to(device)
            text_embeddings = model.encode_text(texts)  #embed with text encoder
            text_embeddings /= text_embeddings.norm(dim=-1, keepdim=True)
            per_descriptor_embeds.append(text_embeddings)

        per_descriptor_embeds = torch.stack(per_descriptor_embeds, dim=1).squeeze()

    if feature_type == "fused":
        similarity_matrix = fused.to(device) @ per_descriptor_embeds.T
    elif feature_type == "distilled":
        similarity_matrix = distill.to(device) @ per_descriptor_embeds.T
    elif feature_type == "ensembled":
        pred_fusion = fused.to(device) @ per_descriptor_embeds.T
        pred_distill = distill.to(device) @ per_descriptor_embeds.T
        feat_ensemble = distill.clone().half()
        mask_ = pred_distill.max(dim=-1)[0] < pred_fusion.max(dim=-1)[0]
        feat_ensemble[mask_] = fused_f[mask_]
        similarity_matrix = feat_ensemble.to(device) @ per_descriptor_embeds.T
        
    if similarity_matrix.ndim == 2:
        if agg_type == "mean":
            agg_sim_mat = torch.mean(similarity_matrix, dim=1)
        elif agg_type == "max":
            agg_sim_mat, _ = torch.max(similarity_matrix, dim=1)
        elif agg_type == "median":
            agg_sim_mat, _ = torch.median(similarity_matrix, dim=1)
        elif agg_type == "quantile":
            agg_sim_mat = torch.quantile(similarity_matrix.float(), quantile, dim=1)
        elif agg_type == "min":
            agg_sim_mat, _ = torch.min(similarity_matrix, dim=1)
        elif agg_type == "bare-overlay-weight-max":
            maximum, _ = similarity_matrix[:,1:].max(dim=0)
            weight = maximum / maximum.sum()
            agg_sim_mat = similarity_matrix[:,0] + weight[None,:] @ similarity_matrix[:,1:].T 
        elif agg_type == "bare-overlay-mean":
            weight = None
            agg_sim_mat = similarity_matrix[:,0] + similarity_matrix[:,1:].mean(axis=1)
        else:
            raise NotImplementedError()
    else: 
        agg_sim_mat = similarity_matrix
        
    if norm:
        agg_sim_mat -= agg_sim_mat.mean()
        
    agg_sim_mat = agg_sim_mat.reshape(-1, 1)
    
    # creating pc
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np.asarray(fpc))
    pcd.colors = o3d.utility.Vector3dVector(np.asarray(fpcc))

    # heatmap
    cmap = plt.get_cmap('bwr')

    # normalize the tensor to the range [0, 1]
    print(f'Min: {torch.min(agg_sim_mat)}')
    print(f'Max: {torch.max(agg_sim_mat)}')
    normalized_tensor = scale * agg_sim_mat + 0.5
    #normalized_tensor = (agg_sim_mat - torch.min(agg_sim_mat)) / (torch.max(agg_sim_mat) - torch.min(agg_sim_mat))

    colors = cmap(normalized_tensor.detach().cpu().numpy().squeeze())
    pcd_heatmap = o3d.geometry.PointCloud()

    pcd_heatmap.points = o3d.utility.Vector3dVector(pcd.points)
    pcd_heatmap.colors = o3d.utility.Vector3dVector(colors[:, :3])

    #transform heatmap to the side
    
    #pcd_heatmap.points = o3d.utility.Vector3dVector(np.asarray(pcd.points) + [0,10,0])
    #o3d.visualization.draw_geometries([pcd, pcd_heatmap])
    
    pcd_heatmap.points = pcd.points
    if draw:
        #o3d.visualization.draw_plotly([pcd_heatmap])
        o3d.visualization.draw_geometries([pcd_heatmap])
    
    return agg_sim_mat

In [ ]:
# load all the required data
#source_path = "/home/aleks/3dcv/openseg_aug/chair_scene_alex/scannet_3d/example/scene0000_00_vh_clean_2.pth"
#fused_path = "/home/aleks/3dcv/openseg_aug/chair_scene_alex/fused/scene0000_00_0.pt"
#distilled_path = "/home/aleks/3dcv/openseg_aug/chair_scene_alex/features_3D/scene0000_00_vh_clean_2_openscene_feat_distill.npy"

source_path = "/home/aleks/3dcv/openseg_aug_new/scannet_3d/example/scene0000_00_vh_clean_2.pth"
fused_path = "/home/aleks/3dcv/openseg_aug_new/fused/scene0000_00_0.pt"
distilled_path = "/home/aleks/3dcv/openseg_aug_new/features_3D/scene0000_00_vh_clean_2_openscene_feat_distill.npy"

source_points, source_colors, source_labels = load_scene(source_path, False)

fused_f, filtered_pc, filtered_pc_c, filtered_pc_labels, indices = load_fused_features(fused_path,
                                                                              source_points, 
                                                                              source_colors,
                                                                              source_labels)
distilled_f = load_distilled_features(distilled_path, indices)

In [ ]:
descriptors = {
    "stool": [
        "Stool with backless",
        "Backless stool",
        "Stool with cylindrical legs",
        "Small stool",
        "Stool with flat seat",
        "Simple stool",
        "Stool with no armrests",
        "Wooden stool",
        "Stool with round seat",
        "Metal stool",
    ],
    "armchair": [
        "Armchair with padded arms",
        "Comfy armchair",
        "Armchair with high back",
        "Upholstered armchair",
        "Armchair with wooden frame",
        "Leather armchair",
        "Armchair with cushioned seat",
        "Modern armchair",
        "Armchair with decorative legs",
        "Vintage armchair",
    ],
    "rocking chair": [
        "Rocking chair with curved runners",
        "Wooden rocking chair",
        "Rocking chair with slat back",
        "Outdoor rocking chair",
        "Upholstered rocking chair",
        "Rocking chair with armrests",
        "Classic rocking chair",
        "Rocking chair with wide seat",
        "Modern rocking chair",
        "Nursery rocking chair",
    ],
    "ball chair": [
        "Round ball chair",
        "Modern ball chair",
        "High back ball chair",
        "Hanging ball chair",
        "Swivel base ball chair",
        "Outdoor ball chair",
        "Padded seat ball chair",
        "Enclosed ball chair",
        "Sphere-shaped ball chair",
        "Retro ball chair",
    ],
}

In [ ]:
descriptors_bare = {
    "stool": ["backless", "compact", "straight legs", "round seat", "simple design", 
              "no armrests", "low height", "versatile", "footrest", "wooden"],
    
    "armchair": ["upholstered", "armrests", "comfortable", "cushioned", "high back", 
                 "wingback", "padded seat", "elegant design", "curved legs", "fabric"],
    
    "rocking chair": ["curved runners", "sloping back", "rocking motion", "wooden frame", "comfortable seat", 
                      "traditional design", "armrests", "relaxing", "classic", "country-style"],
    
    "ball chair": ["sphere-shaped", "modern", "futuristic", "enclosed space", "swivel base", 
                   "comfortable cushion", "unique design", "bubble chair", "acrylic material", "iconic"]
}

In [ ]:
SCANNET_LABELS_20 = ['wall', 'floor', 'cabinet', 'bed', 'chair', 
                     'sofa', 'table', 'door', 'window', 'bookshelf', 
                     'picture','counter', 'desk', 'curtain', 'refrigerator', 
                     'shower curtain', 'toilet', 'sink', 'bathtub', 'otherfurniture']

SCANNET_LABELS_AUG = SCANNET_LABELS_20 + list(descriptors.keys())

UNKNOWN_ID = 255
NO_FEATURE_ID = 256

agg_type = 'mean'
feature_type = 'fused'

# Visualize similarity

In [ ]:
feature_type = 'ensembled'
obj_class = 'ball chair'

sim_class = highlight_query(['stool'], feature_type, 'mean', distilled_f, fused_f, filtered_pc, 
                            filtered_pc_c, device, draw=True, scale=2.)

sim_class = highlight_query(['armchair'], feature_type, 'mean', distilled_f, fused_f, filtered_pc, 
                            filtered_pc_c, device, draw=True, scale=2.)

#sim_desc  = highlight_query(descriptors['armchair'], feature_type, 'mean', 
#                            distilled_f, fused_f, filtered_pc, filtered_pc_c, device, draw=True, scale=2.)

#draw_improvement(sim_class, sim_desc, filtered_pc)

# Similarity Results for Descriptors

In [ ]:
similarity_results = {}
difference_results = {}

In [ ]:
for class_name in SCANNET_LABELS_AUG:
    if class_name not in similarity_results.keys():
        similarity_results[class_name] = highlight_query([class_name], feature_type, agg_type, 
                                                         distilled_f, fused_f, filtered_pc, 
                                                         filtered_pc_c, device, draw=False)

for class_name, descriptor_list in descriptors.items():
    for descriptor in descriptor_list:
        if descriptor not in similarity_results.keys():
            similarity = highlight_query([descriptor], feature_type, agg_type, 
                                         distilled_f, fused_f, filtered_pc, 
                                         filtered_pc_c, device, draw=False)
            similarity_results[descriptor] = similarity
            difference_results[descriptor] = similarity - similarity_results[class_name]

In [ ]:
similarity_results_bare = {}
difference_results_bare = {}

In [ ]:
for class_name in SCANNET_LABELS_AUG:
    if class_name not in similarity_results_bare.keys():
        similarity_results_bare[class_name] = similarity_results[class_name]

for class_name, descriptor_list in descriptors_bare.items():
    for descriptor in descriptor_list:
        if descriptor not in similarity_results_bare.keys():
            similarity = highlight_query([descriptor], feature_type, agg_type, 
                                         distilled_f, fused_f, filtered_pc, 
                                         filtered_pc_c, device, draw=False)
            similarity_results_bare[descriptor] = similarity
            difference_results_bare[descriptor] = similarity - similarity_results_bare[class_name]

# Generate Graphs

In [ ]:
for obj_class, descs_class in descriptors.items():
    gt_label = SCANNET_LABELS_AUG.index(obj_class)
    other_chair_mask = np.all([filtered_pc_labels != gt_label, 
                               20 <= filtered_pc_labels, 
                               filtered_pc_labels < len(SCANNET_LABELS_AUG)], axis=0)
    
    y_labels = []
    x_similarity_other_general = []
    x_difference_other_general = []
    
    x_similarity_other_chair = []
    x_difference_other_chair = []
    
    
    sim = similarity_results[obj_class]
        
    avg_similarity_class = sim[filtered_pc_labels == gt_label].mean().cpu()
    avg_similarity_other_chair = sim[other_chair_mask].mean().cpu()
    avg_similarity_other_general = sim[filtered_pc_labels != gt_label].mean().cpu()

    y_labels.append(obj_class)

    x_similarity_other_general.append(avg_similarity_class-avg_similarity_other_general)
    x_similarity_other_chair.append(avg_similarity_class-avg_similarity_other_chair)
    
    for desc in descs_class:
        sim = similarity_results[desc]
        
        avg_similarity_class = sim[filtered_pc_labels == gt_label].mean().cpu()
        avg_similarity_other_chair = sim[other_chair_mask].mean().cpu()
        avg_similarity_other_general = sim[filtered_pc_labels != gt_label].mean().cpu()
        
        diff = difference_results[desc]
        
        avg_difference_class = diff[filtered_pc_labels == gt_label].mean().cpu()
        avg_difference_other_chair = diff[other_chair_mask].mean().cpu()
        avg_difference_other_general = diff[filtered_pc_labels != gt_label].mean().cpu()
        
        y_labels.append(desc)
        
        x_similarity_other_general.append(avg_similarity_class-avg_similarity_other_general)
        x_difference_other_general.append(avg_difference_class-avg_difference_other_general)
        x_similarity_other_chair.append(avg_similarity_class-avg_similarity_other_chair)
        x_difference_other_chair.append(avg_difference_class-avg_difference_other_chair)

    sorted_indices = sorted(range(len(y_labels)), key=lambda i: x_similarity_other_chair[i])
        
    plt.figure(figsize=(8,4))
    Y_axis = np.arange(len(y_labels))
    plt.barh(Y_axis-0.2, [x_similarity_other_general[i] for i in sorted_indices], 0.4, color='blue',
             label='to other objects')
    plt.barh(Y_axis+0.2, [x_similarity_other_chair[i] for i in sorted_indices], 0.4, color='orange',
             label='to other chairs')
    plt.yticks(Y_axis, [y_labels[i] for i in sorted_indices])
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1))
    plt.title(f'Average similarity difference of \'{obj_class}\'')


print('Done!')

In [ ]:
for obj_class, descs_class in descriptors_bare.items():
    gt_label = SCANNET_LABELS_AUG.index(obj_class)
    other_chair_mask = np.all([filtered_pc_labels != gt_label, 
                               20 <= filtered_pc_labels, 
                               filtered_pc_labels < len(SCANNET_LABELS_AUG)], axis=0)
    
    y_labels = []
    x_similarity_other_general = []
    x_difference_other_general = []
    
    x_similarity_other_chair = []
    x_difference_other_chair = []
    
    
    sim = similarity_results_bare[obj_class]
        
    avg_similarity_class = sim[filtered_pc_labels == gt_label].mean().cpu()
    avg_similarity_other_chair = sim[other_chair_mask].mean().cpu()
    avg_similarity_other_general = sim[filtered_pc_labels != gt_label].mean().cpu()

    y_labels.append(obj_class)

    x_similarity_other_general.append(avg_similarity_class-avg_similarity_other_general)
    x_similarity_other_chair.append(avg_similarity_class-avg_similarity_other_chair)
    
    for desc in descs_class:
        sim = similarity_results_bare[desc]
        
        avg_similarity_class = sim[filtered_pc_labels == gt_label].mean().cpu()
        avg_similarity_other_chair = sim[other_chair_mask].mean().cpu()
        avg_similarity_other_general = sim[filtered_pc_labels != gt_label].mean().cpu()
        
        diff = difference_results_bare[desc]
        
        avg_difference_class = diff[filtered_pc_labels == gt_label].mean().cpu()
        avg_difference_other_chair = diff[other_chair_mask].mean().cpu()
        avg_difference_other_general = diff[filtered_pc_labels != gt_label].mean().cpu()
        
        y_labels.append(desc)
        
        x_similarity_other_general.append(avg_similarity_class-avg_similarity_other_general)
        x_difference_other_general.append(avg_difference_class-avg_difference_other_general)
        x_similarity_other_chair.append(avg_similarity_class-avg_similarity_other_chair)
        x_difference_other_chair.append(avg_difference_class-avg_difference_other_chair)

    sorted_indices = sorted(range(len(y_labels)), key=lambda i: x_similarity_other_chair[i])
        
    plt.figure(figsize=(8,4))
    Y_axis = np.arange(len(y_labels))
    plt.barh(Y_axis-0.2, [x_similarity_other_general[i] for i in sorted_indices], 0.4, 
             label='to other objects', color='blue')
    plt.barh(Y_axis+0.2, [x_similarity_other_chair[i] for i in sorted_indices], 0.4, 
             label='to other chairs', color='orange')
    plt.yticks(Y_axis, [y_labels[i] for i in sorted_indices])
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1))
    plt.title(f'Average similarity difference of \'{obj_class}\'')


print('Done!')

In [ ]:
for _descriptors in various_descriptors:
    
    SCANNET_LABELS_DESC = SCANNET_LABELS_20.copy()
    for key, value in _descriptors.items():
        SCANNET_LABELS_DESC.append(value)
        
    print(SCANNET_LABELS_DESC)
    print(f'Aggregation: {agg_type}')
    print(f'Features: {feature_type}')
        
    class_ious, class_accs, mean_iou, mean_acc, confusion = \
        evaluate(SCANNET_LABELS_DESC, _descriptors, feature_type, agg_type , distilled_f, fused_f, filtered_pc_labels)
    
    print(f'Mean acc: {mean_acc}')
    
    col_sums = confusion.sum(axis=0)
    col_sums[col_sums==0] = 1
    confusion = confusion / col_sums[np.newaxis, :]
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(confusion)
    fig.colorbar(cax)
    
    #ax.set_xticks(np.arange(len(SCANNET_LABELS_AUG)), labels=SCANNET_LABELS_AUG)
    ax.set_yticks(np.arange(len(SCANNET_LABELS_AUG)), labels=SCANNET_LABELS_AUG)
    #plt.setp([tick.label for tick in ax.xaxis.get_major_ticks()], rotation=45, ha="left",
    #     rotation_mode="anchor")

    plt.show()
    
    print_results_table(SCANNET_LABELS_DESC, class_ious, _descriptors)

In [ ]:
print_results_table(SCANNET_LABELS_20, class_ious, descriptors)